In [2]:
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import numpy as np

# Baseline product

In [ ]:
# Combine table and format datetime
# paths:
base_path = "/home/datasets4/stein"
data_root = f"{base_path}/jena_experiment_data_raw/jena_experiment"
output_root = f"{base_path}/jena_experiment_data_various_products/covariates_processed"
stack = []
mypath = f"{data_root}/moisture"
onlyfiles = [f.split("D")[0] for f in listdir(mypath)]
for folder in onlyfiles: 
   p = mypath +  "/" + folder + "Dataset" + "/" + folder + "data.csv"
   stack.append(pd.read_csv(p, on_bad_lines='skip', sep= ";"))
# group for the sample and rake the mean
for x in range(len(stack)):
    # extract hour for meaning
    # remove extreme % values (only up because drought seems to be possible under 5)
    problem = stack[x][["Vol10","Vol20","Vol30","Vol40","Vol60","Vol100"]] > 70
    print(problem.sum())
    stack[x][problem] = np.nan 
    stack[x]["time"] = pd.to_datetime(stack[x]["time"]).dt.hour
    stack[x] = stack[x].drop(columns = ["sample", "comment"]).groupby(["plot", "date"]).mean(numeric_only=True).reset_index()
    #reformat to time to add it to the datetime stamp.
    stack[x]["time"] = pd.to_datetime(stack[x]["time"], format="%H")
    #update the date with the mean hour
    stack[x]["datetime"] = stack[x]["date"] + " " +  stack[x]["time"].astype(str).str[-8:]
    stack[x] = stack[x].drop(columns = ["date", "time"])

sm = pd.concat(stack).sort_values("datetime").reset_index(drop=True)
# reorder columns
sm = sm[["datetime", "plot","Vol10","Vol20","Vol30","Vol40","Vol60","Vol100"]]
sm.to_csv(f"{output_root}/moisture_base.csv", index=False)

Vol10     0
Vol20     0
Vol30     0
Vol40     0
Vol60     4
Vol100    5
dtype: int64
Vol10     0
Vol20     0
Vol30     0
Vol40     0
Vol60     0
Vol100    7
dtype: int64
Vol10      0
Vol20      0
Vol30      0
Vol40      5
Vol60      0
Vol100    14
dtype: int64
Vol10       3
Vol20       1
Vol30       1
Vol40      12
Vol60     207
Vol100    573
dtype: int64
Vol10     1
Vol20     0
Vol30     0
Vol40     3
Vol60     0
Vol100    1
dtype: int64
Vol10       0
Vol20       0
Vol30       0
Vol40       0
Vol60       0
Vol100    745
dtype: int64


# Advanced products

In [6]:
moisture = pd.read_csv(f"{output_root}/moisture_base.csv")
moisture["datetime"] = pd.to_datetime(moisture["datetime"])
moisture.sort_values("datetime", inplace = True)
moisture["week"] = moisture["datetime"].dt.week
moisture["year"] = moisture["datetime"].dt.year
moisture.drop(columns= ["datetime"])

# mean for events where measurements were taken on multiple days in the week. 
moisture = moisture.groupby(["year", "week", "plot"]).mean().reset_index()


/tmp/ipykernel_2663960/4286036191.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  moisture["week"] = moisture["datetime"].dt.week


In [7]:
# ignore winter months:  (10 Because most years start roughly at least there with measurements)
m = moisture[(moisture["week"] >=  13) & (moisture["week"] <=  43)]
# Remove 2002 because its measured very late and plant div is not establised anyways
m = m[m["year"] != 2002]

In [9]:
# Now we need to fill potential missing points.
# First we calculate the mean for each time stamp and try to fill with this as good as it is possible.
# After this we use the seasonal mean of each plot.
meansV = m.groupby(["week", "year"]).mean().reset_index()
meansV = m[["year", "week", "plot"]].merge(meansV, on= ["year", "week"], how = "left")
meansH = m[m.columns[1:]].groupby(["week", "plot"]).mean().reset_index()
meansH = m[["year", "week", "plot"]].merge(meansH, on= ["week", "plot"], how = "left")

In [ ]:
# Fill with week mean
print((m.isnull().sum() / len(m))["Vol10"])
for x in m.columns: 
    index = m[x].isnull().values
    m.loc[index, x] = meansV.loc[index, x].values
print((m.isnull().sum() / len(m))["Vol10"])
# If there are small week gaps we can interpolate directly. 
m = m.interpolate(limit = 1)
print((m.isnull().sum() / len(m))["Vol10"])

# Remaining with seasonal mean
for x in m.columns: 
    index = m[x].isnull().values
    m.loc[index, x] = meansH.loc[index, x].values
print((m.isnull().sum() / len(m))["Vol10"])

# Fill the tiny rest for which no seasonal mean exists
m = m.interpolate(limit = 1)
print((m.isnull().sum() / len(m))["Vol10"])


m.to_csv(f"{output_root}/moisture_weekly_filled.csv", index=False)


0.048802543683089755
0.010501443948542924
0.010443102593273242
0.0
0.0


In [ ]:
# yearly product.
m = pd.read_csv(f"{output_root}/moisture_weekly_filled.csv")
m = m.drop(columns="week")
yearly = m.groupby(["year", "plot"])
mean = yearly.mean()
std = yearly.std()
stab = mean / std

mean.columns = [x + "_mean" for x in mean.columns]
std.columns = [x + "_std" for x in std.columns]
stab.columns = [x + "_stab" for x in stab.columns]

yearly = pd.concat([mean, std, stab], axis = 1).reset_index()
yearly.to_csv(f"{output_root}/moisture_yearly.csv", index=False)